In [19]:

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from numpy import ndarray
from enum import Enum
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score

data = load_breast_cancer()
X, y = data.data, data.target

# 80 - 10 - 10 split into train test validate
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

# use standard scaler to normalize the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train = PolynomialFeatures(degree=2, include_bias=True).fit_transform(X_train)
X_val = PolynomialFeatures(degree=2, include_bias=True).fit_transform(X_val)
X_test = PolynomialFeatures(degree=2, include_bias=True).fit_transform(X_test)


class Node():
    def __init__(self, data, feature_idx, feature_val, prediction_probs, information_gain) -> None:
        self.data = data
        self.feature_idx = feature_idx
        self.feature_val = feature_val
        self.prediction_probs = prediction_probs
        self.information_gain = information_gain
        self.feature_importance = self.data.shape[0] * self.information_gain
        self.left = None
        self.right = None


class RandomForest():
    def __init__(self, n_tree=10,  min_samples_leaf=1, min_information_gain=0.0) -> None:
        self.n_tree = n_tree
        self.min_samples_leaf = min_samples_leaf
        self.min_information_gain = min_information_gain

    def feature_importance(self, trees):
        """ calcs avg feature importance for random forest"""
        tree_feature_importance = []
        for tree in trees:
            
            tree_feature_importance.append(tree.feature_importances)

        all_feature_importances = [list(x.values()) for x in tree_feature_importance]
        average_feature_importance = np.mean(all_feature_importances, axis=0)

        return average_feature_importance
    
    def train(self, X_train: np.array, Y_train: np.array) -> None:        
            self.tree_list = []
            for _ in range(self.n_tree):
                tree = DecisionTree(min_samples_leaf=self.min_samples_leaf,
                                            min_information_gain=self.min_information_gain)
                tree.train(X_train, Y_train)
                self.tree_list.append(tree)

            self.feature_importances = self.feature_importance(self.tree_list)
    
    def predict(self, X_set: np.array):
            pred_probs = []
            trees_pred_probs = []

            for tree in self.tree_list:
                trees_pred_probs.append(tree.predict_tree(X_set))

            for i in range(X_set.shape[0]):
                tree_probs_for_i = [a[i] for a in trees_pred_probs]
                # calc average for each index
                average_pred_probs = np.mean(tree_probs_for_i, axis=0)
                pred_probs.append(average_pred_probs)
            print (pred_probs)
            preds = np.argmax(pred_probs, axis=1)
            return preds

class DecisionTree():
    def __init__(self, min_samples_leaf=1, 
                 min_information_gain=0.0) -> None:
        self.min_samples_leaf = min_samples_leaf
        self.min_information_gain = min_information_gain

    def entropy(self, class_probabilities: list): 
        return sum([-p * np.log2(p) for p in class_probabilities if p>0])
    
    def labels_probabilities(self, labels: np.array):
        total_count = len(labels)
        unique_labels, label_counts = np.unique(labels, return_counts=True)
        return label_counts / total_count

    def group_entropy(self, subsets: np.array):
        total_count = len(subsets) 
        return sum([ self.entropy(self.labels_probabilities(subset)) * (len(subset) / total_count) for subset in subsets])
    
    def split(self, data: np.array, feature_idx: int, feature_val: float):   
        threshold = data[:, feature_idx] < feature_val
        a = data[threshold]
        b = data[~threshold]
        return a, b
        
    def get_split(self, data: np.array): ## gets the best split (by entropy)
        min_part_entropy = 1e9
        feature_idx_to_use = list(range(data.shape[1]-1))

        for idx in feature_idx_to_use:
            feature_vals = np.percentile(data[:, idx], q=np.arange(25, 100, 25))
            for feature_val in feature_vals:
                g1, g2, = self.split(data, idx, feature_val)
                part_entropy = self.group_entropy([g1[:, -1], g2[:, -1]])
                if part_entropy < min_part_entropy:
                    min_part_entropy = part_entropy
                    min_entropy_feature_idx = idx
                    min_entropy_feature_val = feature_val
                    g1_min, g2_min = g1, g2
        return g1_min, g2_min, min_entropy_feature_idx, min_entropy_feature_val, min_part_entropy

    def find_y_probabilities(self, data: np.array) -> np.array:
        total_labels = len(data[:,-1].astype(int))
        label_probabilities = np.zeros(len(self.labels_in_train), dtype=float) 
        for i, label in enumerate(self.labels_in_train):
            label_index = np.where(data[:,-1].astype(int) == i)[0]
            if len(label_index) > 0:
                label_probabilities[i] = len(label_index) / total_labels
        return label_probabilities
    
    def train(self, X_train: np.array, Y_train: np.array):
        self.labels_in_train = np.unique(Y_train)
        train_data = np.concatenate((X_train, np.reshape(Y_train, (-1, 1))), axis=1)
        self.tree = self.create_tree(data=train_data, current_depth=0)
        self.feature_importances = dict.fromkeys(range(X_train.shape[1]), 0)
        self.feature_importance(self.tree)
        # normalize features importances
        self.feature_importances = {k: v / total for total in (sum(self.feature_importances.values()),) for k, v in self.feature_importances.items()}


    def create_tree(self, data: np.array, current_depth: int):
        left_data, right_data, split_feature_idx, split_feature_val, split_entropy = self.get_split(data) # find best split
        label_probabilities = self.find_y_probabilities(data)     # fibd label probs for the node
        node_entropy = self.entropy(label_probabilities) # calc information gain
        information_gain = node_entropy - split_entropy

        node = Node(data, split_feature_idx, split_feature_val, label_probabilities, information_gain)

        if self.min_samples_leaf > left_data.shape[0] or self.min_samples_leaf > right_data.shape[0]:
            return node
        elif information_gain < self.min_information_gain:
            return node
        current_depth += 1
        node.left = self.create_tree(left_data, current_depth)
        node.right = self.create_tree(right_data, current_depth)
        
        return node
    
    def predict_tree(self, X_set: np.array):
        def _predict_one_sample(X: np.array):
            node = self.tree
            while node:
                pred_probs = node.prediction_probs
                if X[node.feature_idx] < node.feature_val:
                    node = node.left
                else:
                    node = node.right
            return pred_probs
        pred_probs = np.apply_along_axis(_predict_one_sample, 1, X_set)

        return pred_probs
    
    def predict(self, X_set: np.array) -> np.array:
        pred_probs = self.predict_tree(X_set)
        preds = np.argmax(pred_probs, axis=1)
        return preds    
    
    def feature_importance(self, node):
        if node != None:
            self.feature_importances[node.feature_idx] += node.feature_importance
            self.feature_importance(node.left)
            self.feature_importance(node.right)    


rf_classifier = RandomForest(n_tree=10, min_samples_leaf=1, min_information_gain=0.0)
rf_classifier.train(X_train, y_train)
y_train_rf = rf_classifier.predict(X_train)

accuracy_rf = accuracy_score(y_train, y_train_rf)

print("Random Forest Training Accuracy:", accuracy_rf)     

[array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.37142857, 0.62857143]), array([0.